In [24]:
import gurobipy as gp
from gurobipy import GRB

In [25]:
arcs, costs = gp.multidict({
    ('Denver', 'Kansas'):     2,
    ('Denver', 'Louisville'): 3,
    #
    ('Atlanta', 'Kansas'):    3,
    ('Atlanta', 'Louisville'): 1,
    #
    ('Kansas', 'Detroit'):    2,
    ('Kansas', 'Miami'):      6,
    ('Kansas', 'Dallas'):     3,
    ('Kansas', 'NewOrleans'): 6,
    #
    ('Louisville', 'Detroit'):    4,
    ('Louisville', 'Miami'):      4,
    ('Louisville', 'Dallas'):     6,
    ('Louisville', 'NewOrleans'): 5,
    
})

warehouses = {'Kansas', 'Louisville'}

supply = {
    'Denver':  600,
    'Atlanta': 400,
}

demand = {
    'Detroit':    200,
    'Miami':      150, 
    'Dallas':     350,
    'NewOrleans': 300, 
}

In [26]:
m = gp.Model('transhipment_problem')

In [27]:
sent = m.addVars(arcs, vtype=GRB.CONTINUOUS, name='send')

In [28]:
Z = sent.prod(costs)
m.ModelSense = GRB.MINIMIZE
m.setObjective(Z)

In [29]:
source = m.addConstrs((sent.sum(plant, '*') <= s for plant, s in supply.items() ), name='source')


transshipment = m.addConstrs(
    (sent.sum(w,'*')  - sent.sum('*', w) == 0 for w in warehouses),
                                        name='transhipment')

sink = m.addConstrs(
    (sent.sum('*', retailer) == d for retailer, d in demand.items()),
                               name='sink')

In [30]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x2748ae01
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 4 rows, 8 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5500000e+03   1.250000e+02   0.000000e+00      0s
       3    5.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.200000000e+03


In [31]:
print(f'Objective Value: ${m.ObjVal}')


Objective Value: $5200.0


In [32]:
print('Decisions:')
for v in m.getVars():
    if abs(v.x) > 0.00001:
        print(v.varName, v.x)


Decisions:
send[Denver,Kansas] 600.0
send[Atlanta,Louisville] 400.0
send[Kansas,Detroit] 200.0
send[Kansas,Dallas] 350.0
send[Kansas,NewOrleans] 50.0
send[Louisville,Miami] 150.0
send[Louisville,NewOrleans] 250.0


In [33]:
m.getConstrs()

[<gurobi.Constr source[Denver,600]>,
 <gurobi.Constr source[Atlanta,400]>,
 <gurobi.Constr transhipment[Louisville]>,
 <gurobi.Constr transhipment[Kansas]>,
 <gurobi.Constr sink[Detroit,200]>,
 <gurobi.Constr sink[Miami,150]>,
 <gurobi.Constr sink[Dallas,350]>,
 <gurobi.Constr sink[NewOrleans,300]>]